<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/gravitational_lensing_pipeline_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
gravitational_lensing_pipeline.py

End-to-end, single-script implementation of a multi-agent ideation pipeline
that maps gravitational lensing concepts into quantum circuit constructs,
synthesizes a (QFT + phase) “lensing” circuit, and critiques it.

- No external services required.
- If Qiskit is available, it uses it; otherwise, it falls back to a pseudo-circuit.
- Prints tracker-style logs and structured JSON artifacts for each stage.
"""

from __future__ import annotations
import json
import math
import random
import sys
from dataclasses import dataclass, field, asdict
from typing import Any, Dict, List, Optional, Tuple, Counter

# -------------------------------
# Optional Qiskit import (soft)
# -------------------------------
try:
    from qiskit import QuantumCircuit
    from qiskit.circuit.library import QFT
    QISKIT_AVAILABLE = True
except Exception:
    QISKIT_AVAILABLE = False


# ===============================
# Utility: pretty JSON
# ===============================
def jprint(title: str, obj: Any) -> None:
    print(f"\n=== {title} ===")
    print(json.dumps(obj, indent=2, ensure_ascii=False))


# ===============================
# Schemas (dataclasses)
# ===============================
@dataclass
class ConceptNode:
    id: str
    label: str
    type: str


@dataclass
class ConceptEdge:
    source: str
    target: str
    relation: str


@dataclass
class ConceptGraph:
    nodes: List[ConceptNode]
    edges: List[ConceptEdge]

    def to_json(self) -> Dict[str, Any]:
        return {
            "nodes": [asdict(n) for n in self.nodes],
            "edges": [asdict(e) for e in self.edges],
        }


@dataclass
class DisambiguatedConcepts:
    concepts: List[str]
    notes: str = ""

    def to_json(self) -> Dict[str, Any]:
        return {"concepts": self.concepts, "notes": self.notes}


@dataclass
class RankedAnalogy:
    analogy: str
    similarity_score: float


@dataclass
class RankedAnalogies:
    source_concepts: List[str]
    rankings: List[RankedAnalogy]

    def to_json(self) -> Dict[str, Any]:
        return {
            "source_concepts": self.source_concepts,
            "rankings": [
                {"analogy": r.analogy, "similarity_score": r.similarity_score}
                for r in self.rankings
            ],
        }


@dataclass
class ModelSpec:
    model_class: str
    description: str
    quantum_map: Dict[str, str]
    num_qubits: int
    params: Dict[str, Any] = field(default_factory=dict)

    def to_json(self) -> Dict[str, Any]:
        return {
            "model_class": self.model_class,
            "description": self.description,
            "quantum_map": self.quantum_map,
            "num_qubits": self.num_qubits,
            "params": self.params,
        }


@dataclass
class QuantumCircuitOutput:
    backend: str  # "qiskit" or "pseudo"
    diagram: str
    gate_counts: Dict[str, int]
    depth: int
    python_code_hint: str  # small how-to-recreate snippet
    metadata: Dict[str, Any] = field(default_factory=dict)

    def to_json(self) -> Dict[str, Any]:
        return {
            "backend": self.backend,
            "diagram": self.diagram,
            "gate_counts": self.gate_counts,
            "depth": self.depth,
            "python_code_hint": self.python_code_hint,
            "metadata": self.metadata,
        }


@dataclass
class CritiqueReport:
    score: int  # 0..100
    issues: List[str]
    recommendations: List[str]
    metrics: Dict[str, Any]

    def to_json(self) -> Dict[str, Any]:
        return {
            "score": self.score,
            "issues": self.issues,
            "recommendations": self.recommendations,
            "metrics": self.metrics,
        }


# ===============================
# Agent Implementations
# ===============================
class IdeaKernelAgent:
    """Generates an initial concept graph from a seed topic."""

    def run(self, seed: str) -> ConceptGraph:
        # Minimal, interpretable concept graph aligned to your domain
        nodes = [
            ConceptNode(id="c1", label="Gravitational Lensing", type="Concept"),
            ConceptNode(id="p1", label="Spacetime Curvature", type="Phenomenon"),
            ConceptNode(id="o1", label="Convex Lens", type="OpticalSystem"),
            ConceptNode(id="q1", label="Quantum Interference", type="Concept"),
            ConceptNode(id="qc1", label="Phase Shift", type="QuantumConstruct"),
            ConceptNode(id="qg1", label="QFT", type="QuantumGate"),
            ConceptNode(id="g1", label="Geodesics", type="Concept"),
            ConceptNode(id="f1", label="Fourier Transform", type="Concept"),
        ]
        edges = [
            ConceptEdge(source="c1", target="q1", relation="IS_ANALOGOUS_TO"),
            ConceptEdge(source="p1", target="qc1", relation="MAPPED_TO"),
            ConceptEdge(source="o1", target="qg1", relation="INSPIRES"),
            ConceptEdge(source="g1", target="q1", relation="RELATES_TO"),
            ConceptEdge(source="f1", target="qg1", relation="IMPLEMENTS_AS"),
        ]
        return ConceptGraph(nodes=nodes, edges=edges)


class ConceptDisambiguator:
    """Normalizes and selects the relevant canonical terms for search/analogy."""

    def run(self, graph: ConceptGraph) -> DisambiguatedConcepts:
        # Choose the core conceptual anchors, normalizing case.
        raw = [
            n.label for n in graph.nodes
            if n.type in {"Concept", "Phenomenon", "OpticalSystem"}
        ]
        canonical = sorted({r.strip().lower() for r in raw})
        notes = "Lowercased and deduplicated; filtered to core semantic categories."
        return DisambiguatedConcepts(concepts=canonical, notes=notes)


class AnalogyRanker:
    """
    Simulates a Neo4j-backed analogy ranking using embedded graph seeds.
    It mirrors the Cypher intent by returning top analogies with scores.
    """

    def __init__(self) -> None:
        # In-memory seeds akin to your Neo4j triples
        self.analogies = {
            "gravitational lensing": [("Quantum Interference", 0.95)],
            "spacetime curvature": [("Phase Shift", 0.90)],
            "convex lens": [("QFT", 0.88)],
            "geodesics": [("Entangled Path Interference", 0.86)],
            "fourier transform": [("QFT", 0.92)],
        }

    def run(self, disambig: DisambiguatedConcepts) -> RankedAnalogies:
        scores: Dict[str, float] = {}
        for c in disambig.concepts:
            for a, s in self.analogies.get(c, []):
                scores[a] = max(scores.get(a, 0.0), s)

        # Add a small deterministic shuffle for ties (but keep reproducible)
        random.seed(42)
        ranked = sorted(
            [RankedAnalogy(analogy=a, similarity_score=s) for a, s in scores.items()],
            key=lambda r: (-r.similarity_score, r.analogy),
        )
        # Keep top 5 like the Cypher LIMIT
        ranked = ranked[:5]
        return RankedAnalogies(source_concepts=disambig.concepts, rankings=ranked)


class PrototypeTrigger:
    """Chooses the best analogy and emits a concrete model spec for synthesis."""

    def run(self, ranked: RankedAnalogies, num_qubits: int = 4) -> ModelSpec:
        top = ranked.rankings[0].analogy if ranked.rankings else "Quantum Interference"
        quantum_map = {
            "gravitational lensing": "quantum phase shift",
            "geodesics": "entangled path interference",
            "optics": "Fourier transforms",
        }
        description = (
            "Use QFT and phase gates to simulate curved spacetime lensing on qubit paths. "
            f"Primary analogy: {top}."
        )
        # A simple “curvature” profile controls per-qubit phase rotations
        phase_profile = [round((i - (num_qubits - 1) / 2) * 0.25, 4) for i in range(num_qubits)]
        params = {
            "use_qft": True,
            "entangle_paths": True,
            "phase_profile": phase_profile,
        }
        return ModelSpec(
            model_class="QiskitTemplate",
            description=description,
            quantum_map=quantum_map,
            num_qubits=num_qubits,
            params=params,
        )


# ===============================
# Circuit synthesis
# ===============================
class CircuitSynthesizer:
    """
    Builds a quantum “lensing” circuit:
      - Optional Hadamard layer (multipath)
      - QFT
      - Phase shifts per qubit (curvature)
      - Inverse QFT
      - Optional entangling CP chain (geodesic coupling)
    Falls back to a pseudo-circuit if Qiskit is unavailable.
    """

    def run(self, spec: ModelSpec) -> QuantumCircuitOutput:
        if QISKIT_AVAILABLE:
            return self._run_qiskit(spec)
        else:
            return self._run_pseudo(spec)

    def _build_phase_profile(self, spec: ModelSpec) -> List[float]:
        prof = spec.params.get("phase_profile")
        if prof is None:
            n = spec.num_qubits
            prof = [(i - (n - 1) / 2) * 0.2 for i in range(n)]
        return [float(x) for x in prof]

    def _run_qiskit(self, spec: ModelSpec) -> QuantumCircuitOutput:
        n = spec.num_qubits
        qc = QuantumCircuit(n, name="LensingSim")

        # Multipath superposition
        for q in range(n):
            qc.h(q)

        if spec.params.get("use_qft", True):
            qc.append(QFT(num_qubits=n, do_swaps=False).to_gate(label="QFT"), qc.qubits)

        # Curvature as phase shifts
        phases = self._build_phase_profile(spec)
        for q, phi in enumerate(phases):
            qc.p(phi, q)

        if spec.params.get("use_qft", True):
            qc.append(QFT(num_qubits=n, do_swaps=False, inverse=True).to_gate(label="QFT†"), qc.qubits)

        # Entangle geodesics with a CP chain
        if spec.params.get("entangle_paths", True):
            for q in range(n - 1):
                qc.cp(0.3, q, q + 1)

        # Basic metrics
        try:
            diagram = qc.draw(output="text").single_string()
        except Exception:
            diagram = str(qc)

        try:
            counts = {k: int(v) for k, v in qc.count_ops().items()}
        except Exception:
            counts = {}

        try:
            depth = qc.depth()
        except Exception:
            depth = -1

        python_hint = (
            "from qiskit import QuantumCircuit\n"
            "from qiskit.circuit.library import QFT\n"
            f"qc = QuantumCircuit({n})\n"
            "for q in range({n}): qc.h(q)\n"
            "qc.append(QFT(num_qubits={n}, do_swaps=False).to_gate(label='QFT'), qc.qubits)\n"
            f"# phase_profile = {phases}\n"
            "for q, phi in enumerate(phase_profile): qc.p(phi, q)\n"
            "qc.append(QFT(num_qubits={n}, do_swaps=False, inverse=True).to_gate(label='QFT†'), qc.qubits)\n"
            "for q in range({n}-1): qc.cp(0.3, q, q+1)\n"
        ).replace("{n}", str(n))

        return QuantumCircuitOutput(
            backend="qiskit",
            diagram=diagram,
            gate_counts=counts,
            depth=depth,
            python_code_hint=python_hint,
            metadata={"phase_profile": phases},
        )

    def _run_pseudo(self, spec: ModelSpec) -> QuantumCircuitOutput:
        # A minimal representation to keep the pipeline runnable without Qiskit
        n = spec.num_qubits
        phases = self._build_phase_profile(spec)
        ops: List[str] = []

        # Superposition
        for q in range(n):
            ops.append(f"H q{q}")

        if spec.params.get("use_qft", True):
            ops.append(f"QFT({n})")

        for q, phi in enumerate(phases):
            ops.append(f"P({phi:.4f}) q{q}")

        if spec.params.get("use_qft", True):
            ops.append(f"QFT†({n})")

        if spec.params.get("entangle_paths", True):
            for q in range(n - 1):
                ops.append(f"CP(0.3) q{q}->q{q+1}")

        diagram = "PseudoCircuit:\n  " + "\n  ".join(ops)
        counts = {}
        for op in ops:
            gate = op.split()[0]
            counts[gate] = counts.get(gate, 0) + 1

        depth = len(ops)

        python_hint = (
            "# Qiskit not available. Pseudo-steps to reproduce:\n"
            f"# qubits: {n}\n"
            "# for q in range(n): H(q)\n"
            "# QFT(n)\n"
            f"# phase_profile = {phases}\n"
            "# for (q, phi) in enumerate(phase_profile): P(phi, q)\n"
            "# QFT†(n)\n"
            "# for q in range(n-1): CP(0.3, q, q+1)\n"
        )

        return QuantumCircuitOutput(
            backend="pseudo",
            diagram=diagram,
            gate_counts=counts,
            depth=depth,
            python_code_hint=python_hint,
            metadata={"phase_profile": phases},
        )


# ===============================
# Critic
# ===============================
class CriticAgent:
    """Scores circuit soundness, complexity, and alignment with the analogy."""

    def run(self, circuit: QuantumCircuitOutput, spec: ModelSpec, ranked: RankedAnalogies) -> CritiqueReport:
        issues: List[str] = []
        recs: List[str] = []

        # Heuristics for evaluation
        has_qft = ("QFT" in circuit.diagram)
        has_phases = ("P(" in circuit.diagram) or (" p(" in circuit.diagram) or ("p" in circuit.gate_counts)
        has_entangle = ("CP(" in circuit.diagram) or ("cp" in circuit.gate_counts)

        # Coverage checks
        if not has_qft:
            issues.append("Missing QFT; the optical-lens analogy may be underrepresented.")
            recs.append("Insert QFT + inverse QFT bracketing the phase shifts.")
        if not has_phases:
            issues.append("No phase shifts detected; cannot represent curvature.")
            recs.append("Add per-qubit phase gates driven by a curvature profile.")
        if not has_entangle:
            recs.append("Consider adding a CP chain to encode geodesic coupling.")

        # Complexity checks
        depth = circuit.depth
        if depth > 150:
            issues.append("Circuit depth is high; may reduce fidelity on NISQ devices.")
            recs.append("Reduce QFT size or sparsify phase profile.")

        # Alignment with analogy
        top_analogy = ranked.rankings[0].analogy if ranked.rankings else "Quantum Interference"
        if "Interference" not in top_analogy and not has_entangle:
            recs.append("Strengthen interference motifs via entanglement or additional Hadamards.")

        # Score synthesis: start from 100 and penalize
        score = 100
        if not has_qft:
            score -= 25
        if not has_phases:
            score -= 30
        if depth > 150:
            score -= 10
        score = max(0, min(100, score))

        metrics = {
            "backend": circuit.backend,
            "depth": depth,
            "gate_counts": circuit.gate_counts,
            "top_analogy": top_analogy,
            "phase_profile": circuit.metadata.get("phase_profile"),
        }
        return CritiqueReport(score=score, issues=issues, recommendations=recs, metrics=metrics)


# ===============================
# Tracker log (optional)
# ===============================
def tracker_log(step: str, status: str, note: str = "") -> None:
    print(f"[TRACK] {step:<22} | {status:<10} | {note}")


# ===============================
# Orchestration
# ===============================
def run_pipeline(
    seed: str = "gravitational lensing",
    num_qubits: int = 4,
) -> Tuple[ConceptGraph, DisambiguatedConcepts, RankedAnalogies, ModelSpec, QuantumCircuitOutput, CritiqueReport]:
    tracker_log("IdeaKernelAgent", "START", f"seed={seed}")
    m1 = IdeaKernelAgent().run(seed)
    tracker_log("IdeaKernelAgent", "DONE", f"nodes={len(m1.nodes)} edges={len(m1.edges)}")

    tracker_log("ConceptDisambiguator", "START", "")
    m2 = ConceptDisambiguator().run(m1)
    tracker_log("ConceptDisambiguator", "DONE", f"concepts={len(m2.concepts)}")

    tracker_log("AnalogyRanker", "START", "")
    m3 = AnalogyRanker().run(m2)
    tracker_log("AnalogyRanker", "DONE", f"ranked={len(m3.rankings)}")

    tracker_log("PrototypeTrigger", "START", f"num_qubits={num_qubits}")
    m4 = PrototypeTrigger().run(m3, num_qubits=num_qubits)
    tracker_log("PrototypeTrigger", "DONE", f"model={m4.model_class}")

    tracker_log("CircuitSynthesizer", "START", f"backend={'qiskit' if QISKIT_AVAILABLE else 'pseudo'}")
    m5 = CircuitSynthesizer().run(m4)
    tracker_log("CircuitSynthesizer", "DONE", f"depth={m5.depth}")

    tracker_log("CriticAgent", "START", "")
    m6 = CriticAgent().run(m5, m4, m3)
    tracker_log("CriticAgent", "DONE", f"score={m6.score}")

    return m1, m2, m3, m4, m5, m6


# ===============================
# Main
# ===============================
def main():
    # CLI-ish defaults
    seed = "gravitational lensing"
    num_qubits = 4

    # Allow quick overrides: python script.py 6  (sets num_qubits=6)
    if len(sys.argv) >= 2:
        try:
            num_qubits = int(sys.argv[1])
        except ValueError:
            pass

    print("\n== gravitational-lensing-quantum-pipeline ==")
    if not QISKIT_AVAILABLE:
        print("Note: Qiskit not detected. Using pseudo-circuit backend. Install qiskit for real circuits.")

    m1, m2, m3, m4, m5, m6 = run_pipeline(seed=seed, num_qubits=num_qubits)

    # Output artifacts
    jprint("ConceptGraph.json", m1.to_json())
    jprint("DisambiguatedConcepts.json", m2.to_json())
    jprint("RankedAnalogies.json", m3.to_json())
    jprint("ModelSpec.json", m4.to_json())
    jprint("QuantumCircuitOutput.json", m5.to_json())
    jprint("CritiqueReport.json", m6.to_json())

    print("\nPipeline complete.")


if __name__ == "__main__":
    main()